**SQL**

**Цель иследования:**

Проанализировать базу данных и сформировать ценностное предложение для нового продукта

**Задания:**

1. Посчитать, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитать количество обзоров и среднюю оценку;
3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;
4. Определить автора с самой высокой средней оценкой книг — учитывая только книги с 50 и более оценками;
5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
              'pwd': 'Sdf4$2;d-d30pp', # пароль
              'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
              'port': 6432, # порт подключения
              'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 
# напишем функцию для удобства
def resrequest(x):
  resrequest = pd.io.sql.read_sql(querty, con=engine)
  return resrequest

### Шаг 1. Исследуем таблицы с данными.

**Таблица books**

In [2]:
# создаём запрос
querty = '''
SELECT *
FROM books
LIMIT 5;
'''
# получаем данные
resrequest(querty)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Таблице `books` содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица authors**

In [3]:
# создаем запрос
querty = '''
SELECT *
FROM authors
LIMIT 5;
'''
# получим данные
resrequest(querty)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**Таблица `authors` cодержит данные об авторах:**

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица publishers**

In [4]:
querty = '''
SELECT *
FROM publishers
LIMIT 5;
'''

resrequest(querty)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


**Таблица `publishers` cодержит данные об издательствах:**

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица ratings**

In [5]:
querty = '''
SELECT *
FROM ratings
LIMIT 5;
'''

resrequest(querty)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


**Таблица `ratings` cодержит данные о пользовательских оценках книг:**

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица reviews**

In [6]:
querty = '''
SELECT *
FROM reviews
LIMIT 5;
'''

resrequest(querty)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Таблица `reviews` cодержит данные о пользовательских обзорах на книги:**

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

**Вывод:** 

Все таблицы соответсвуют описанию, никаких ошибок замечено не было.

### Шаг 2. Посчитайем, сколько книг вышло после 1 января 2000 года

In [7]:
querty2 = '''
SELECT count(book_id) as book
FROM books
WHERE publication_date > '2000/01/01';
'''
request = pd.io.sql.read_sql(querty2, con=engine)
request

,book
0,819


**Вывод:**

После 1 января 2000 года вышло 819 книг.

### Для каждой книги посчитаем количество обзоров и среднюю оценку

In [8]:
request = '''
SELECT title AS books, review, average_estimate 
FROM books LEFT JOIN 
            (SELECT book_id, 
                    COUNT(review_id) AS review FROM reviews GROUP BY book_id) AS count_rew
                    ON count_rew.book_id = books.book_id 
            LEFT JOIN 
                    (SELECT book_id, 
                    ROUND(AVG(rating)) AS average_estimate FROM ratings GROUP BY book_id) AS avg_rating
                    ON books.book_id = avg_rating.book_id;
'''
request2 = pd.io.sql.read_sql(request, con=engine)
request2

,books,review,average_estimate
0,The Body in the Library (Miss Marple #3),2.0,5.0
1,Galápagos,2.0,5.0
2,A Tree Grows in Brooklyn,5.0,4.0
3,Undaunted Courage: The Pioneering First Missio...,2.0,4.0
4,The Prophet,4.0,4.0
...,...,...,...
995,Alice in Wonderland,4.0,4.0
996,A Woman of Substance (Emma Harte Saga #1),2.0,5.0
997,Christine,3.0,3.0
998,The Magicians' Guild (Black Magician Trilogy #1),2.0,4.0


### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц.

In [9]:
query3 = '''
SELECT
    publishers.publisher,
    COUNT(book_id)
FROM books INNER JOIN publishers ON books.publisher_id = publishers.publisher_id
WHERE books.num_pages > 50
GROUP BY publishers.publisher
ORDER BY count DESC
LIMIT 1;
'''
request3 = pd.io.sql.read_sql(query3, con = engine)
request3

,publisher,count
0,Penguin Books,42


**Вывод:**

Издательство Penguin Books выпустило 42 книги толще 50 страниц. Это издательство выпустило наибольшее количество книг толще 50 страниц.

### Определим автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [10]:
query4 = '''
SELECT
    author,
    ROUND(AVG(subq.rating), 2) AS avg_rating
FROM (
    SELECT
        author,
        title,
        AVG(rating) AS rating
    FROM (
        SELECT *
        FROM books
        WHERE book_id IN (
            SELECT book_id
            FROM ratings
            GROUP BY book_id
            HAVING COUNT(rating) >= 50
        ) 
    ) AS rated_books
    INNER JOIN authors ON rated_books.author_id = authors.author_id
    INNER JOIN ratings ON rated_books.book_id = ratings.book_id
    GROUP BY author, title
    ORDER BY author
) AS subq
GROUP BY author
ORDER BY avg_rating DESC
LIMIT 1
'''
request4 = pd.io.sql.read_sql(query4, con = engine)
request4

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.28


**Вывод:**

Автор, которым поставили самые высокие оценки за книги, толще 50 страниц - на первом месте Джоанн Роулинг и иллюстратор Мэри Гранпрэ.

### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [11]:
# создаём запрос
query5 = '''
SELECT
   ROUND(AVG(subq.review_count)) AS average_review_count
FROM (
    SELECT COUNT(review_id) AS review_count
        FROM
            reviews
        WHERE username IN (
            SELECT
                username
            FROM ratings
            GROUP BY username
            HAVING COUNT(rating) > 50
            )
    GROUP BY username
    ) AS subq;
'''
request5 = pd.io.sql.read_sql(query5, con = engine)
request5

,average_review_count
0,24.0


**Вывод:**

Среднее количество обзоров от пользователей, которые поставили больше 50 оценок составило 24.

### Общий вывод

Проанализировав базы данных было выявленно:

- После 1 января 2000 года вышло 819 книг. 
- Издательство Penguin Books выпустило 42 книги толще 50 страниц. Это издательство выпустило наибольшее количество книг толще 50 страниц.
- Автор, которому поставили самые высокие оценки за книги, толще 50 страниц - Джоанн Роулинг и иллюстратора Мэри Гранпрэ.
- Среднее количество обзоров от пользователей, которые поставили больше 50 оценок составило 24.